Import TensorFlow into your program to get started:

In [9]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [10]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


Build a tf.keras.Sequential model:

In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [13]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.3726684 ,  0.1925742 , -0.21753323, -0.36477917, -0.36610174,
        -0.31367803, -0.09629193, -0.02162172, -0.5664115 , -0.5252636 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [14]:
tf.nn.softmax(predictions).numpy()

array([[0.16815025, 0.14043766, 0.09319148, 0.08043186, 0.08032555,
        0.08464884, 0.10520361, 0.11335991, 0.06574463, 0.06850631]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [16]:
loss_fn(y_train[:1], predictions).numpy()

2.4692438

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [17]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate our model

Use the `Model.fit` method to adjust our model parameters and minimize the loss: 

In [18]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2975 - accuracy: 0.9142
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1433 - accuracy: 0.9580
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1073 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0884 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0734 - accuracy: 0.9772


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [19]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0716 - accuracy: 0.9794 - 577ms/epoch - 2ms/step


[0.07162024080753326, 0.9793999791145325]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [20]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [22]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.1570718e-08, 1.3397782e-09, 5.8246349e-07, 1.1657822e-04,
        2.3253042e-11, 1.4353915e-09, 3.4617349e-14, 9.9988139e-01,
        5.5751446e-08, 1.3913029e-06],
       [5.3149142e-07, 3.9387724e-04, 9.9918264e-01, 1.4448455e-04,
        2.8135345e-15, 1.4965585e-06, 1.5414521e-07, 5.0652008e-14,
        2.7698828e-04, 8.6231092e-12],
       [9.2394998e-08, 9.9678952e-01, 1.1114710e-04, 1.7216169e-05,
        7.4833719e-05, 2.9003400e-06, 4.0591408e-06, 2.3300499e-03,
        6.6891860e-04, 1.2083233e-06],
       [9.9918240e-01, 5.1758394e-07, 1.7449314e-04, 2.8209408e-06,
        8.8230103e-05, 2.4738927e-05, 3.5625603e-04, 8.7700173e-05,
        6.3359889e-06, 7.6442469e-05],
       [1.0629543e-06, 2.6146881e-07, 2.0349516e-06, 1.7736625e-08,
        9.9500483e-01, 1.1597900e-06, 1.2923126e-06, 4.0298441e-05,
        1.8785267e-05, 4.9302271e-03]], dtype=float32)>

We have trained a machine learning model using a prebuilt dataset using the Keras API.